# Imports

In [45]:
import pandas as pd
import numpy as np

In [46]:
# Importation des csv 
df_vf = pd.read_csv("../csv_clean/valeurs_foncieres_2024.csv")
df_vf_old = pd.read_csv("../csv_clean/valeurs_foncieres_2023.csv")
df_parcs = pd.read_csv("../csv_clean/parcs_jardins_communes_agg.csv")
df_college = pd.read_csv("../csv_clean/colleges.csv")
df_lycee = pd.read_csv("../csv_clean/lycees.csv")

# Tries besoins

In [47]:
# besoins 
nbr_piece = 4 # minimum 
rapport_qualite_prix = 0.75 #% minimum 
investissement = 0.5

# qualité 
surface = 70 # minimum 
score_vert = 75 #% minimum 
ecole = 5 # minimum

In [48]:
dict_24 = df_vf.groupby("nom_commune")["valeur_fonciere"].mean().to_dict()
dict_20 = df_vf_old.groupby("nom_commune")["valeur_fonciere"].mean().to_dict()

In [49]:
commune_names = []
bad_inv_name = []

for key_24, key_20 in zip(dict_24, dict_20):
    if dict_24[key_24] / dict_20[key_20] < investissement:
        bad_inv_name.append(key_24)
    else:
        commune_names.append(key_24)

print(bad_inv_name)
print(len(bad_inv_name), '/', len(dict_24), '\n')

['azolette', 'bron', 'brullioles', 'chassieu', 'chaussan', 'chénas', 'cogny', 'courzieu', 'dardilly', 'dième', 'genas', 'givors', 'juliénas', 'la mulatière', 'lantignié', 'les haies', 'liergues', 'limas', 'longessaigne', 'lyon 1er arrondissement', 'lyon 5e arrondissement', 'marchampt', 'meaux-la-montagne', 'meys', 'millery', 'moiré', 'montromant', 'orliénas', 'pomeys', 'propières', 'ranchal', 'rivolet', 'rontalon', 'régnié-durette', 'saint-appolinaire', 'saint-bonnet-des-bruyères', 'saint-clément-sur-valsonne', 'saint-cyr-le-chatoux', 'saint-didier-sur-beaujeu', "saint-genis-l'argentière", 'saint-genis-laval', 'saint-georges-de-reneins', 'saint-igny-de-vers', "saint-just-d'avray", 'saint-laurent-de-chamousset', "saint-nizier-d'azergues", 'saint-pierre-la-palud', 'saint-romain-en-gier', 'sainte-colombe', "sainte-foy-l'argentière", 'tarare', 'ternand', 'thizy-les-bourgs', 'thurins', 'tupin-et-semons', 'valsonne', 'vauxrenard', 'vernay', 'villefranche-sur-saône', 'vindry-sur-turdine', 'vo

In [50]:
def filtred_df(df):
    # join (concaténations) des collèges et lycées 
    df_ecoles = pd.concat([df_college, df_lycee])
    df_ecoles["nb_ecoles"] = df_ecoles.groupby("commune")["commune"].transform("count")

    # join (merge) les valeurs foncières et les parcs
    df = df.merge(df_parcs[["code_commune", "score_espaces_verts"]], on="code_commune", how="left")

    # join (merge) de dataframe et les écoles 
    df = df.merge(df_ecoles[["code_postal", "nb_ecoles"]], on="code_postal", how="left")

    # sup les communes à mauvais investissement
    df = df[~df["nom_commune"].isin(bad_inv_name)]

    # call function to filter with room and quality/price
    df = filtered_room_quality_rpice(df)
    
    return df

In [51]:
def filtered_room_quality_rpice(df):
    # définir qualité pour faire rapport qualité/prix
    df["qualite"] = np.where(
        (df["surface_reelle_bati"] >= surface) & (df["score_espaces_verts"] >= score_vert) & (df["nb_ecoles"] >= ecole),
        1,  # très bonne qualité
        np.where(
            (df["surface_reelle_bati"] >= surface) | (df["score_espaces_verts"] >= score_vert) | (df["nb_ecoles"] >= ecole),
            0.5,  # qualité moyenne
            0   # faible qualité
        )
    )
    print('Qualité moyenne :', df["qualite"].mean())

    # calcul du prix moyen dans la région lyonnaise 
    prix_moyen_lyon = df["valeur_fonciere"].mean()
    print('Prix moyen de Lyon :', round(prix_moyen_lyon))

    # qualité / prix 
    df["qualite_prix"] = (df["qualite"] / df["valeur_fonciere"]) * prix_moyen_lyon
    print('Rapport qualité prix moyen :', df["qualite_prix"].mean())

    print('Nombre pièces moyennes :', df["nombre_pieces_principales"].mean())
    print('Nombre pièces max :', df["nombre_pieces_principales"].max())

    # filtrage des données 
    filtred_df = df[
        (df["nombre_pieces_principales"] >= nbr_piece) &
        (df["qualite_prix"] >= rapport_qualite_prix)
    ].copy()

    print(filtred_df.shape)
    return filtred_df

In [52]:
df24 = filtred_df(df_vf)
print('\n', df24["nom_commune"].unique())


Qualité moyenne : 0.45671153413481763
Prix moyen de Lyon : 590294
Rapport qualité prix moyen : 2028.3825140461042
Nombre pièces moyennes : 1.2531119860787414
Nombre pièces max : 28.0
(31355, 16)

 ['lyon 4e arrondissement' 'oullins' 'loire-sur-rhône' 'rillieux-la-pape'
 'bessenay' 'fontaines-sur-saône' 'caluire-et-cuire' "l'arbresle"
 'beauvallon' 'saint-germain-nuelles' 'lyon 9e arrondissement' 'quincieux'
 "champagne-au-mont-d'or" 'savigny' 'francheville' 'cublize'
 'porte des pierres dorées' 'gleizé' 'tassin-la-demi-lune' 'brignais'
 'belleville-en-beaujolais' 'anse' 'sainte-foy-lès-lyon' 'vernaison'
 'blacé' 'saint-jean-la-bussière' 'lozanne' 'chamelet' 'écully' 'mornant'
 'condrieu' 'irigny' 'cours' 'lyon 2e arrondissement'
 'rochetaillée-sur-saône' 'odenas' 'villechenève' 'grandris' 'bully'
 'quincié-en-beaujolais' 'sathonay-camp' 'saint-romain-en-gal' 'lentilly'
 'craponne' 'genay' "saint-marcel-l'éclairé" 'poule-les-écharmeaux'
 'brindas' 'grigny' 'saint-symphorien-sur-coise' '

In [53]:
df24.to_csv("../csv_clean/filtered_valeurs_foncieres_2024.csv", index=False)